# Comment faire un TER en 5 min

In [1]:
library(devtools)
library(slam)
library(skmeans)
library(rngtools)
library(mclust)
library(raster)
library(schoolmath)
#library(nmfgpu4R)
library(NbClust)
library(ggplot2)
library(factoextra)
library(ggplot2)
#library(NMF)

Loading required package: pkgmaker
Loading required package: registry

Attaching package: ‘pkgmaker’

The following object is masked from ‘package:base’:

    isNamespaceLoaded

Package 'mclust' version 5.4
Type 'citation("mclust")' for citing this R package in publications.
Loading required package: sp
Welcome! Related Books: `Practical Guide To Cluster Analysis in R` at https://goo.gl/13EFCZ


In [2]:
Sys.setenv(CUDA_PATH="/usr/local/cuda-9.1/")
Sys.setenv(NMFGPU_ROOT="/usr/local/nmfgpu/") 

In [ ]:
nmfgpu4R.init(quiet = F)

In [3]:
getwd()
setwd("/home/mbenhamd/Project/nmf-ter/")

[1] "/home/mbenhamd/Project/nmf-ter"

In [4]:
data = read.csv("matrice-greyscale-16.csv")
dim(data)

[1] 4777  256

In [ ]:
library(nmfgpu4R) 

In [ ]:

results = nmf(t(data), r= 71, algorithm="mu",initMethod="EIn-NMF",runs=1,maxiter=10000,threshold=0.1, thresholdType="rmsd",verbose=T)

In [ ]:
show_images = function(data,from=1,to=10){
  final=vector()
for (variable in seq(from,to)) {
  b = matrix(data[,variable],nrow=16,ncol=16,byrow = T)
  final=cbind(final,b)  
}
  rst <- as.raster(final,max=max(final))
  plot(rst)
}

In [ ]:
#show_images(as.matrix(data),1,5)
show_images(results$W%*%results$H,1,8)

In [5]:
d = as.matrix(as.data.frame(lapply(data, as.numeric)))

In [ ]:
dyn.load("libKMCUDA.so")
res = .External("kmeans_cuda", d,10, tolerance=0.01, verbosity=0, average_distance=TRUE)  

In [ ]:
res

In [ ]:
v = vector()
for (variable in seq(from=10,to=100)){
  result = .External("kmeans_cuda", d,variable, tolerance=0.01, verbosity=0, average_distance=TRUE)  
  v=cbind(v,result$average_distance)  
}


In [6]:
library(NMF)

Loading required package: cluster
NMF - BioConductor layer [NO: missing Biobase] | Shared memory capabilities [OK] | Cores 7/8
  To enable the Bioconductor layer, try: install.extras('
NMF
') [with Bioconductor repository enabled]


In [ ]:
nmfSeed()

In [ ]:
# Use a seed that will be set before each first run
res <- nmf(t(d), 47:57, method='KL', nrun=1, seed="nndsvd",.options='v4tp8')

Compute NMF rank= 47  ... 

NMF algorithm: 'KL'
# ** Current RNG settings:


# RNG kind:  Mersenne-Twister / Inversion 
# RNG state: 403L, 98L, ..., 762760408L [87ef45d8a7f925dd9e44db1db181c950] 


# Setting up RNG ... 
# RNG setup: standard [using current RNG]
NMF seeding method: nndsvd
Warning message in sqrt(S[i] * termp) * uup:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termp) * vvp:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termp) * uup:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termp) * vvp:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termn) * uun:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termn) * vvn:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”War

”Warning message in sqrt(S[i] * termn) * uun:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termn) * vvn:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termn) * uun:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termn) * vvn:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termp) * uup:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termp) * vvp:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termn) * uun:
“Recycling array of length 1 in array-vector ari

...

+ measures ... OK
Compute NMF rank= 50  ... 

NMF algorithm: 'KL'
# ** Current RNG settings:


# RNG kind:  Mersenne-Twister / Inversion 
# RNG state: 403L, 98L, ..., 762760408L [87ef45d8a7f925dd9e44db1db181c950] 


# Setting up RNG ... 
# RNG setup: standard [using current RNG]
NMF seeding method: nndsvd
Warning message in sqrt(S[i] * termp) * uup:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termp) * vvp:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termp) * uup:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termp) * vvp:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termn) * uun:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termn) * vvn:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”War

”Warning message in sqrt(S[i] * termn) * uun:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termn) * vvn:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termn) * uun:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termn) * vvn:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termp) * uup:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termp) * vvp:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in sqrt(S[i] * termn) * uun:
“Recycling array of length 1 in array-vector ari

In [ ]:
plot(res)

In [ ]:
png('test.png',width = 1920, height = 1080)
plot(res)
dev.off()